<a href="https://colab.research.google.com/github/IshitaSharma0/MedGemma-Diagnostic-Assistant/blob/main/medgemma_diagnosis_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.2 MB/s eta 0:00:00


In [ ]:
from transformers import BitsAndBytesConfig, AutoProcessor, AutoModelForImageTextToText, pipeline
from PIL import Image
import torch
import gradio as gr


In [ ]:
from huggingface_hub import notebook_login, get_token
if get_token() is None:
    notebook_login()

In [ ]:
model_variant = "4b-it"
model_id = f"google/medgemma-{model_variant}"
use_quantization = True

In [ ]:
# Model load config
model_kwargs = dict(
    torch_dtype=torch.bfloat16,
    # torch_dtype=torch.float32,
    device_map="auto",
)

if use_quantization:
    model_kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True)

In [ ]:
# Load processor and model
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    **model_kwargs,
)
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "image-text-to-text",
    model=model,
    processor=processor,
    model_kwargs=model_kwargs,
)

pipe.model.generation_config.do_sample = False

Device set to use cuda:0


In [ ]:
# Inference function for Gradio
def medgemma_infer(uploaded_image, prompt):
    if not uploaded_image or not prompt.strip():
        return "Please upload an image and enter a prompt."

    messages = [
        {"role": "system", "content": [{"type": "text", "text": "You are an expert radiologist."}]},
        {"role": "user", "content": [
            {"type": "text", "text": prompt},
            {"type": "image", "image": uploaded_image}
        ]}
    ]

    output = pipe(text=messages, max_new_tokens=300)
    response = output[0]["generated_text"][-1]["content"]
    return response


In [14]:
# Launch Gradio
gr.Interface(
    fn=medgemma_infer,
    inputs=[
        gr.Image(type="pil", label="Upload Medical Image"),
        gr.Textbox(lines=2, label="Describe symptoms or findings")
    ],
    outputs=gr.Textbox(label="MedGemma Diagnosis"),
    title="🧠 MedGemma Medical Visual Assistant",
    description="Upload a medical image and describe symptoms. Model will return an expert-like diagnosis."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://696090078db9a1ac2b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
